# 01-ANN

一个简单的人工神经网络。

In [ ]:
# Import here.
import numpy as np
import torch
import collections

from matplotlib import pyplot as plt

import visdom

In [ ]:
# Visdom here.
g_is_visdom_on = True
try:
    g_vis = visdom.Visdom(raise_exceptions=True)
except ConnectionError:
    g_vis = None
    g_is_visdom_on = False

给出标准函数：
$$
f(x) = x^2 + \dfrac{1}{\mathrm e^x - \mathrm e^{-x}} \pod{x > 0}
$$

In [ ]:
def f(x, exp=np.exp):
    return x**2 + 1 / (exp(x) - exp(-x))

In [ ]:
def show_f():
    xs = np.linspace(0.01, 2, 1000)
    plt.plot(xs, f(xs))
    if g_vis:
        g_vis.line(f(xs), xs)
show_f()
del show_f

定义神经网络。

In [ ]:
import net
torch.manual_seed(3)
g_my_net = net.ann()

生成数据集。

In [ ]:
import dataset
g_train_dataset = dataset.make_dataset(f, 200000, 3, 8)
g_test_dataset = dataset.make_dataset(f, 500, 5, 15)

初始化 visdom。

In [ ]:
g_win_loss = [None]
g_win_loss_options = {
    'title': 'Batch Loss Trace',
    'xlabel': 'Batch Count',
    'ylabel': 'Loss'
}

g_win_test = [None]
g_win_test_options = {
    'title': 'Test',
    'xlabel': 'x',
    'ylabel': 'y',
    'markersize': 1
}

初始化测试器。

In [ ]:
import test
def my_test():
    test.test(g_my_net, torch.cuda.is_available(), g_test_dataset, len(
        g_test_dataset), 0, g_vis, g_win_test, g_win_test_options)

训练。

In [ ]:
import train
g_my_net.optimizer.state = collections.defaultdict(dict)
train.train(g_my_net, torch.cuda.is_available(), g_train_dataset,
            1000, 0, 2, g_vis, g_win_loss, g_win_loss_options, my_test)